In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-multi-class-prediction-of-obesity-risk/data.csv
/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


# **肥満リスクの多クラス予測**
(Multi-Class Prediction of Obesity Risk)

# 参考資料
**書籍**


**Kaggle**
- [Obesity! | EDA | XGBoost | Py](https://www.kaggle.com/code/khsamaha/obesity-eda-xgboost-py)
- [[PlayGround S4E2] 📊 EDA + 🤖 Modeling [LightGBM]](https://www.kaggle.com/code/akhiljethwa/playground-s4e2-eda-modeling-lightgbm)

**自分で作成したファイル**
- 銀行解約データセットを使用した二項分類
- 肝硬変の転帰の多クラス予測

**その他**


In [2]:
import numpy as np
import pandas as pd

FILE_PATH = '/kaggle/input/playground-series-s4e2/'
CREATE_DATA = '/kaggle/input/my-multi-class-prediction-of-obesity-risk'


# 確認 (data_import.py)
def file_to_xy(filename):
    data = pd.read_csv(filename, index_col=0)
    print(f'読み込み完了 {filename}')
    
    target_col = 'NObeyesdad'
    
    train = data[:20758].reset_index(drop=True)
    test = data[20758:].reset_index(drop=True).drop(target_col, axis=1) # 目的変数を指定する
    # 目的変数と説明変数に分割
    X = train.drop([target_col], axis=1) # 目的変数を指定する
    y = train[target_col].values # 目的変数を指定する

    return data,test,train,X,y

filename = 'data.csv'
data,test,train,X,y = file_to_xy(CREATE_DATA + '/' + filename)

読み込み完了 /kaggle/input/my-multi-class-prediction-of-obesity-risk/data.csv


In [3]:
# condlist = [train["NObeyesdad"] == "Insufficient_Weight",
#             train["NObeyesdad"] == "Normal_Weight",
#             train["NObeyesdad"] == "Obesity_Type_I",
#             train["NObeyesdad"] == "Obesity_Type_II",
#             train["NObeyesdad"] == "Obesity_Type_III",
#             train["NObeyesdad"] == "Overweight_Level_I",
#             train["NObeyesdad"] == "Overweight_Level_II",
#            ]
# choicelist = [0,1,2,3,4,5,6]

# train["NObeyesdad"] = np.select(condlist, choicelist)

In [4]:
train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,1,1095,933,1839,1,1,265,589,2,0,1491,0,0,1053,2,3,6.0
1,0,108,128,1376,1,1,265,624,1,0,756,0,751,1068,3,0,1.0
2,0,108,1040,1253,1,1,206,127,2,0,659,0,587,1322,3,3,0.0
3,0,522,1035,936,1,1,1053,624,2,0,464,0,1040,806,2,3,4.0
4,1,1605,2050,2205,1,1,735,252,2,0,727,0,1330,995,2,3,6.0


In [5]:
target_col = 'NObeyesdad'

X = train.drop([target_col], axis=1) # 目的変数を指定する
y = train[target_col].values # 目的変数を指定する

# 機械学習モデルの構築・学習・予測

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

sample_sub = pd.read_csv(FILE_PATH + "sample_submission.csv")

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [7]:
# 混同行列
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 機械学習モデルのフォーマットを設定
seed = 42

xgb = XGBClassifier(random_state=seed)
lgb = LGBMClassifier(random_state=seed)
cb = CatBoostClassifier(random_state=seed, verbose=0)

submit = False

## ハイパーパラメータ(optuna)

### Catboost

In [8]:
if submit==False:

    import optuna
    from catboost import CatBoostClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    # Assuming 'X' is your feature matrix and 'y' is your target variable
    X = X.copy()
    y = y.copy()

    def objective(trial):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        params = {
            'iterations': trial.suggest_int('iterations', 200, 1000),
            'depth': trial.suggest_int('depth', 3, 10),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 20),
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.2, log=True),
            'random_state': 42,
            'verbose': 0,
#             'eval_metric': 'Accuracy',
        }

        model = CatBoostClassifier(**params)

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50)

        y_pred = model.predict(X_val)
        metrics = model.score(y_val, y_pred)

        return metrics

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    trial = study.best_trial

    print('Value: ', trial.value)
    print('Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

[I 2024-02-03 07:12:20,203] A new study created in memory with name: no-name-74b44108-0a18-42c1-aecc-af7b16668ad1
[I 2024-02-03 07:12:29,131] Trial 0 finished with value: 0.18762042389210018 and parameters: {'iterations': 869, 'depth': 4, 'min_data_in_leaf': 2, 'learning_rate': 0.00017091219408304231}. Best is trial 0 with value: 0.18762042389210018.
[I 2024-02-03 07:12:34,474] Trial 1 finished with value: 0.12572254335260116 and parameters: {'iterations': 712, 'depth': 3, 'min_data_in_leaf': 12, 'learning_rate': 0.04280698096811655}. Best is trial 0 with value: 0.18762042389210018.
[I 2024-02-03 07:12:59,242] Trial 2 finished with value: 0.12692678227360307 and parameters: {'iterations': 635, 'depth': 8, 'min_data_in_leaf': 20, 'learning_rate': 0.010862323892121974}. Best is trial 0 with value: 0.18762042389210018.
[I 2024-02-03 07:13:06,201] Trial 3 finished with value: 0.12789017341040462 and parameters: {'iterations': 765, 'depth': 5, 'min_data_in_leaf': 12, 'learning_rate': 0.1120

Number of finished trials:  100
Best trial:
Value:  0.18762042389210018
Params: 
    iterations: 869
    depth: 4
    min_data_in_leaf: 2
    learning_rate: 0.00017091219408304231


### XGBClassifier

In [9]:
if submit == False:
    import optuna
    from xgboost import XGBClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score, log_loss

    # Assuming 'X' is your feature matrix and 'y' is your target variable
    X = X.copy()
    y = y.copy()

    def objective(trial):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        params = {
            'objective':"multi:softmax",
            'max_depth': trial.suggest_int('max_depth', 5, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 150, 1000),
            'subsample': trial.suggest_float('subsample', 0.01, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
#             'gamma': trial.suggest_uniform("gamma", 0.05, 0.8),
#             'reg_lambda': trial.suggest_uniform("reg_lambda", 1.3, 2.3),
#             'reg_alpha': trial.suggest_uniform("reg_alpha", 1.5, 2.2),
            'random_state': trial.suggest_categorical('random_state', [42]),
            'tree_method': 'hist',  # Use GPU for training
            'device': 'cuda',
            'njobs': -1,
            'eval_metric': 'logloss',  # Evaluation metric
            'verbosity': 0,  # Set verbosity to 0 for less output
        }

        model = XGBClassifier(**params)

#         model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
        model.fit(X_train, y_train)

        y_pred = model.predict_proba(X_val)
        metrics = log_loss(y_val, y_pred)

        return metrics

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=200)

    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    trial = study.best_trial

    print('Value: ', trial.value)
    print('Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

[I 2024-02-03 07:32:11,499] A new study created in memory with name: no-name-6df2ff63-717a-4873-a518-0e5d2627a8d5
[I 2024-02-03 07:32:16,620] Trial 0 finished with value: 0.32151857921380894 and parameters: {'max_depth': 9, 'min_child_weight': 10, 'learning_rate': 0.47570481820587024, 'n_estimators': 701, 'subsample': 0.7098488661634732, 'colsample_bytree': 0.12476299217268856, 'random_state': 42}. Best is trial 0 with value: 0.32151857921380894.
[I 2024-02-03 07:32:22,174] Trial 1 finished with value: 0.41730679970502155 and parameters: {'max_depth': 10, 'min_child_weight': 7, 'learning_rate': 0.6026432578223785, 'n_estimators': 488, 'subsample': 0.9630179281215326, 'colsample_bytree': 0.8515293583211844, 'random_state': 42}. Best is trial 0 with value: 0.32151857921380894.
[I 2024-02-03 07:32:29,120] Trial 2 finished with value: 0.3242207185265507 and parameters: {'max_depth': 9, 'min_child_weight': 7, 'learning_rate': 0.32658864376022595, 'n_estimators': 989, 'subsample': 0.65667392

Number of finished trials:  200
Best trial:
Value:  0.2636091824869837
Params: 
    max_depth: 6
    min_child_weight: 4
    learning_rate: 0.07295466187890404
    n_estimators: 210
    subsample: 0.3056642077352055
    colsample_bytree: 0.6264624207408099
    random_state: 42


In [10]:
print('Best parameters:', study.best_params)

Best parameters: {'max_depth': 6, 'min_child_weight': 4, 'learning_rate': 0.07295466187890404, 'n_estimators': 210, 'subsample': 0.3056642077352055, 'colsample_bytree': 0.6264624207408099, 'random_state': 42}


### LGBMClassifier

In [11]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report

from optuna.samplers import TPESampler
import optuna

import warnings
warnings.filterwarnings("ignore")

In [12]:
target_col = 'NObeyesdad'

X = train.drop([target_col], axis=1) # 目的変数を指定する
y = train[target_col].values # 目的変数を指定する
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 7,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y_train)
    preds = gbm.predict(X_val)
    accuracy = accuracy_score(y_val, preds)
    
    return accuracy

In [13]:
sampler = TPESampler(seed=42)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=200)

[I 2024-02-03 07:46:21,858] A new study created in memory with name: lightgbm
[I 2024-02-03 07:46:24,859] Trial 0 finished with value: 0.9072736030828517 and parameters: {'lambda_l1': 2.348881295853308e-05, 'lambda_l2': 3.6010467344475403, 'num_leaves': 188, 'feature_fraction': 0.759195090518222, 'bagging_fraction': 0.4936111842654619, 'bagging_freq': 2, 'min_child_samples': 10}. Best is trial 0 with value: 0.9072736030828517.
[I 2024-02-03 07:46:28,149] Trial 1 finished with value: 0.9087186897880539 and parameters: {'lambda_l1': 0.6245760287469893, 'lambda_l2': 0.002570603566117598, 'num_leaves': 182, 'feature_fraction': 0.41235069657748147, 'bagging_fraction': 0.9819459112971965, 'bagging_freq': 6, 'min_child_samples': 25}. Best is trial 1 with value: 0.9087186897880539.
[I 2024-02-03 07:46:31,431] Trial 2 finished with value: 0.901252408477842 and parameters: {'lambda_l1': 4.329370014459266e-07, 'lambda_l2': 4.4734294104626844e-07, 'num_leaves': 79, 'feature_fraction': 0.7148538589

In [14]:
print('Best parameters:', study.best_params)

Best parameters: {'lambda_l1': 1.3019828669773963, 'lambda_l2': 1.0378733814005926e-05, 'num_leaves': 43, 'feature_fraction': 0.5200508885969131, 'bagging_fraction': 0.8788385892118535, 'bagging_freq': 7, 'min_child_samples': 49}


In [15]:
print('Best value:', study.best_value)

Best value: 0.9108863198458574


In [16]:
print('Best trial:', study.best_trial)

Best trial: FrozenTrial(number=36, state=TrialState.COMPLETE, values=[0.9108863198458574], datetime_start=datetime.datetime(2024, 2, 3, 7, 48, 7, 770934), datetime_complete=datetime.datetime(2024, 2, 3, 7, 48, 9, 980287), params={'lambda_l1': 1.3019828669773963, 'lambda_l2': 1.0378733814005926e-05, 'num_leaves': 43, 'feature_fraction': 0.5200508885969131, 'bagging_fraction': 0.8788385892118535, 'bagging_freq': 7, 'min_child_samples': 49}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_freq': IntDistribution(high=7, log=False, low=1, step=1), 'min_child_samples': IntDistribution(high=

### パラメータ

In [17]:
lgb_params = {'lambda_l1': 1.3019828669773963, 
              'lambda_l2': 1.0378733814005926e-05, 
              'num_leaves': 43, 
              'feature_fraction': 0.5200508885969131, 
              'bagging_fraction': 0.8788385892118535, 
              'bagging_freq': 7, 
              'min_child_samples': 49}

xgb_params = {'max_depth': 6, 
              'min_child_weight': 10, 
              'learning_rate': 0.9739305940758409, 
              'n_estimators': 435, 
              'subsample': 0.43576967696348323, 
              'colsample_bytree': 0.025619729678897467, 
              'random_state': 42}

cat_params = {'iterations': 526,
    'depth': 3,
    'min_data_in_leaf': 13,
    'learning_rate': 0.0008029819794568197}

In [18]:
# lgb_model = LGBMClassifier(**lgb_params)
xgb_model = XGBClassifier(**xgb_params)
cat_model = CatBoostClassifier(**cat_params)

# 参考（目的変数のエンコーディング）

In [19]:
# df[target_col] = df[target_col].map({
#     "Insufficient_Weight":0,
#     "Normal_Weight":1,
#     "Obesity_Type_I":2,
#     "Obesity_Type_II":3,
#     "Obesity_Type_III":4,
#     "Overweight_Level_I":5,
#     "Overweight_Level_II":6,
#     })
